## Using AgentChat API with AssistantAgent

In this demo we will be using AgentChat API to create a simple AssistantAgent and will understand its capabilities

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
gpt4o_mini_model = OpenAIChatCompletionClient(model = 'gpt-4o-mini')

## Creating Our First Assistant Agent


In [5]:
assistant = AssistantAgent(
    name = 'my_assistant',
    model_client=gpt4o_mini_model
)

### Testing the Assistant Agent

We'll be using `run` method to send a taks and get a response

In [6]:
result = await assistant.run(task='What is the capitol of India?')

print(result.messages)

[TextMessage(id='53fb67e9-d249-4058-afcc-c30341d6f1eb', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 14, 8, 20, 23, 440290, tzinfo=datetime.timezone.utc), content='What is the capitol of India?', type='TextMessage'), TextMessage(id='d9219928-73aa-4e91-b7e0-844636e9ffd1', source='my_assistant', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=10), metadata={}, created_at=datetime.datetime(2025, 7, 14, 8, 20, 25, 428529, tzinfo=datetime.timezone.utc), content='The capital of India is New Delhi. TERMINATE', type='TextMessage')]


**User Query**

[TextMessage(id='53fb67e9-d249-4058-afcc-c30341d6f1eb', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 14, 8, 20, 23, 440290, tzinfo=datetime.timezone.utc), content='What is the capitol of India?', type='TextMessage'), 

**Assistant Response**

TextMessage(id='d9219928-73aa-4e91-b7e0-844636e9ffd1', source='my_assistant', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=10), metadata={}, created_at=datetime.datetime(2025, 7, 14, 8, 20, 25, 428529, tzinfo=datetime.timezone.utc), content='The capital of India is New Delhi. TERMINATE', type='TextMessage'

### Structure of Agent Response

User query and Assistant reponse both are available as TextMessages. Each message has:

- **id** : A uniqe id defined.
- **source** : User, assistant or tool.
- **model_usage** : It provides the prompts token and completion token used by LLM.
- **metadata** : some more info.
- **created_at** : datetime at which the it was run and response was created.
- **content** : This is the main context which we need. This can be user query or agent response.

Each message has a Source

In [8]:
print(result.messages[-1].content)

The capital of India is New Delhi. TERMINATE
